In [0]:
BEGIN
    DECLARE rows_inserted INT DEFAULT 0;
    DECLARE alerts_created INT DEFAULT 0;
    DECLARE process_date DATE DEFAULT current_date();

    -- Insert cleaned data into metrics table
    INSERT INTO iot_device_metrics
    SELECT
      device_id,
      user_id,
      CAST(timestamp AS TIMESTAMP) AS timestamp,
      num_steps,
      calories_burnt,
      miles_walked
    FROM iot_device_raw
    WHERE device_id IS NOT NULL
      AND num_steps > 0;

    SET rows_inserted = (SELECT COUNT(*) FROM iot_device_metrics);

    -- Aggregate to daily summary
    INSERT OVERWRITE iot_daily_summary
    SELECT
      device_id,
      user_id,
      DATE(timestamp) AS date,
      SUM(num_steps) AS total_steps,
      SUM(miles_walked) AS total_miles,
      SUM(calories_burnt) AS total_calories
    FROM iot_device_metrics
    GROUP BY device_id, user_id, DATE(timestamp);

    -- Create alerts for high activity devices
    INSERT INTO iot_alerts (device_id, alert_type, alert_message, created_at)
    SELECT
      device_id,
      'HIGH_ACTIVITY' AS alert_type,
      CONCAT('Device ', device_id, ' recorded ', total_steps, ' steps on ', date) AS alert_message,
      current_timestamp() AS created_at
    FROM iot_daily_summary
    WHERE total_steps > 15000;

    SET alerts_created = (SELECT COUNT(*) FROM iot_alerts);

    -- Create users with email Ids
    INSERT INTO migrate_to_dbsql.webinar.device_access (device_id, user_email) VALUES
    ('1', 'alice@example.com'),
    ('2', 'bob@example.com'),
    ('3', 'carol@example.com'),
    ('4', 'dave@example.com'),
    ('5', 'abhijit.tilak@databricks.com'),
    ('6', 'eric.begg@databricks.com'),
    ('7', 'zachary.ryan@databricks.com'),
    ('8', 'heidi@example.com'),
    ('9', 'ivan@example.com');
    -- Display summary
    SELECT
      rows_inserted AS metrics_processed,
      alerts_created AS high_activity_alerts,
      process_date AS processing_date;
  END;
